In [ ]:
### QUESTIONS
1. How do we update whether our state has changed?
2. Should I wait until trajectory asks for a behavior? Or should I always be working?

In [1]:
from collections import namedtuple
from math import pi, sqrt, atan2

ILLEGAL_COST = 100
DANGEROUS_COST = 1000

Location = namedtuple( "Location", ["s", "lane"])
Lane     = namedtuple("Lane", ["id", "number"]) # 0 for number means closest to center yellow line
Goal     = Location
Pose     = namedtuple("Pose", ["s", "d", "lane"])
State    = namedtuple("State", ["s", "d", "lane", "yaw"])

class CircleMap(object):
    def __init__(self, radius, num_lanes, lane_width=4.0):
        self.r = radius
        self.num_lanes = num_lanes
        self.lane_width = lane_width
        self.speed_limit = 30.0 # about 65 mph
        
    def get_curvature_at(self, s, d, lane_num):
        r = self.r + lane_num * self.lane_width + d
        return 1.0 / r
    
    def get_speed_limit(self, s, lane_num):
        return self.speed_limit
    
    def to_frenet(self, x, y):
        r = sqrt(x**2 + y**2)
        if r < self.radius:
            lane = 0
        elif r > self.radius * self.num_lanes * self.lane_width:
            lane = self.num_lanes
        else:
            best_lane_distance = 100000000
            best_lane = 0
            for i in range(self.num_lanes):
                ref_rad = self.r + (i + 0.5) * self.num_lanes
                diff = abs(r - ref_rad)
                if diff < best_lane_distance:
                    best_lane_distance = diff
                    best_lane = i
            lane = best_lane
        
        ref_rad = self.radius + (0.5 + lane) * self.lane_width
        d = r - ref_rad
        s = (ref_rad * theta) % (2*pi)
        return Pose(s, d, lane)
    
    def distance_between(self, pose1, pose2):
        """
        returns ds and dd:  driving distance between pose1 and pose2 if
        vehicle1 were to move a distance ds longitudinally in their frame followed 
        by movement of dd laterally
        """
        r1 = self.r + (0.5 + pose1.lane) * self.lane_width + pose1.d
        r2 = self.r + (0.5 + pose2.lane) * self.lane_width + pose2.d
        
        circ1 = r1 * 2 * pi
        circ2 = r2 * 2 * pi
        
        theta1 = pose1.s / circ1
        theta2 = pose2.s / circ2
        
        dtheta = (theta2 - theta1) % (2*pi)
        ds = r1 * dtheta
        dd = r2 - r1
        return ds, dd
        
class Vehicle(object):
    def __init__(self, L, W, max_turn=35.0 * 180.0 / pi, max_acc=9.8):
        self.L = L
        self.W = W
        self.max_turn = max_turn
        self.max_acc = 9.8
        self.pose = None
        self.speed = None
        
    def set_pose(self, pose):
        self.pose = pose
        
    def set_velocity(self, v):
        self.v = v
        
    def set_lane(self, lane):
        self.pose = Pose(self.s, self.d, lane)
    
    @property
    def s(self): return self.pose.s
    
    @property
    def d(self): return self.pose.d
    
    @property
    def lane(self): return self.pose.lane
    
    def _advance(self, dt):
        self.s += self.speed * dt
        
    def _get_coords(self, track):
        r = track.radius + (self.lane_num + 0.5) * track.lane_width
        circum = 2.0 * math.pi * r
#         print "circum is {} and s is {}".format(circum, self.s)
        theta = (2*math.pi * float(self.s % circum) / circum) % (2*math.pi)
        r = r + self.d
        _x = r * math.cos(theta) * -1
        _y = r * math.sin(theta) * -1
        x = _x + CANVAS_WIDTH / 2
        y = _y + CANVAS_HEIGHT  / 2
        return x,y
    
    def update(self, dt, track):
        old_x, old_y = self._get_coords(track)
        self._advance(dt)
        x, y = self._get_coords(track)
        return x, y
PredictedPose = namedtuple("PredictedPose", ["pose", "timestamp"])

class PredictedVehicle(object):
    def __init__(self, data):
        self._id = data["id"]
        self.length = data["length"]
        self.width = data["width"]
        self.predictions = self.process_predictions(data["predictions"])
        
    def process_predictions(self, data):
        new_preds = []
        for p in data:
            pose = Pose(p['s'], p['d'], p['lane'])
            timestamp = pose["timestamp"]
            new_preds.append(PredictedPose(pose, time))
        return tuple(new_preds)

class Decider(object):
    STATES = [
        "ready_to_drive",
        "follow_lane",
        "lane_change_left",
        "lane_change_right",
        "track_gap_left", # used to prepare for a lane change left
        "track_gap_right"
    ]
    
    ALLOWED_TRANSITIONS = [
        [0, 1, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 1],
        [0, 1, 1, 0, 0, 0],
        [0, 1, 0, 1, 0, 0],
        [0, 1, 1, 0, 1, 0],
        [0, 1, 0, 1, 0, 1]
    ]
    
    def __init__(self, vehicle):
        self.ego = vehicle
        self.state = self.STATES[0]
        self.last_suggested_state = None
        self.available_predictions = {}
        self._goal = None
        self.map = None
    
    def set_map(self, M):
        self.map = M
    
    def set_goal(self, goal):
        self._goal = goal
    
    # should be run in background thread
    def _update_with_new_predictions(self, predictions):
        new_vehicles = {}
        for v in predictions["vehicles"]:
            pv = PredictedVehicle(v)
            new_vehicles[pv._id] = pv
        self.available_predictions = new_vehicles
    
    def update_with_new_localization(self, localization):
        x = localization["x"]
        y = localization["y"]
        
        pose = self.map.to_frenet(x,y)
        self.ego.set_pose(pose)
        
    def get_next_states(self):
        idx = self.STATES.index(self.state)
        return [s for s,allowed in zip(self.STATES, self.ALLOWED_TRANSITIONS[idx]) if allowed]
    
    def get_behavior(self):
        possible_next_states = self.get_next_states()
        costs = []
        for state in possible_next_states:
            cost = self.calculate_cost(state)
            costs.append((cost,state))
        _, next_state = min(costs)
        return next_state

    def calculate_cost(state, s, v, a, lane, predictions, goal, MAP):
        # cost functions generally designed to output numbers between -1 and 1
        # unless dangerous or illegal behavior
        cost_functions = [
            self.lane_change_cost,
            self.velocity_cost,
            self.gap_size_cost
        ]
        weights = [
            1,
            1,
            1
        ]
        cost = 0
        for cost_func, weight in zip(cost_functions, weights):
            cost += weight * cost_func(state)
        return cost

    def lane_change_cost(self, state):
        if state not in ["lane_change_left", "lane_change_right"]:
            return 0
        diffs = {"lane_change_left":-1, "lane_change_right":+1}
        next_lane = lane + diffs[state]
        if next_lane >= self.map.num_lanes or next_lane < 0:
            return DANGEROUS_COST
        if abs(self.goal.lane - next_lane) < abs(self.goal.lane - self.ego.lane):
            # next_lane is closer to goal than current lane
            return -1
        else:
            return 1

    def velocity_cost(self, state):
        speed_limit = MAP.get_speed_limit(self.ego.s, self.ego.lane)
        velocity_for_state = self.get_velocity_for_state(self, state)
        if velocity_for_state > speed_limit: 
            return ILLEGAL_COST
        return (speed_limit - velocity_for_state) / speed_limit # between 0 and 1
    
    def get_velocity_for_state(self, state):
        if state == "ready_to_drive": return 0
        SL = self.map.get_speed_limit(self.ego.s, self.ego.lane)
        leading = None
        if state == "follow_lane": 
            leading = self.get_leading_vehicle()
            if not leading:
                return SL
            else:
                return min(self.velocity_for(leading), SL)
        if state.starts_with("lane_change"):
            diffs = {"lane_change_left":-1, "lane_change_right":+1}
            other_lane = self.ego.lane + diffs[state]
            leading = self.get_leading_vehicle(in_lane=other_lane)
            if not leading:
                return SL
            else:
                return min(self.velocity_for(leading), SL)
    
    def get_leading_vehicle(self, in_lane=None):
        if in_lane is None:
            in_lane = self.ego.lane
        pass
    
    def velocity_for(self, predicted_vehicle, in_secs=0):
        pass

    def gap_size_cost(self, state):
        
        L = MAP.vehicle_length

        # next evaluate the cost associated with all the available transitions

        # COSTS INCLUDE: 
        # 1. Does this make progress to the goal 
        #    in terms of number of lane changes remaining. 
        #    Idea is to penalize lane changes if they aren't necessary.
        # 2. Velocity: We want to drive as close to the speed 
        #    limit as possible. (NOTE: Include speed limit in map)
        # 3. Gap size: when making lane changes we prefer to aim for 
        #    larger gaps than smaller ones.
        # 4. If you wanted to go further you might need a  

In [2]:
# NOTE - need to emphasize the weaknesses of cost function approach and
#        cost function design.
#      - may want to send desired vehicle state to trajectory
#      - we had a long discussion about whether or not to include "track gap"
#        state. There is good pedagogical value to the debate that happened.
#        Maybe Benjamin plays the role of "screw it! Add another state!" and
#        Toby wants to keep the design of the state machine simple.



def decide_behavior(MAP, loc, predictions, goal):
    """
    - To be called every 200 - 400 ms.
    - We are NOT responsible for safety. That's the job of trajectory 
      (operating at a 50 ms update rate.)
    """
    if not car.initialized:
        car.state = "ready_to_drive"
    s,v,a,lane = process_localization_data(loc, MAP)
    possible_next_states = get_possible_states(car.state)
    costs = []
    for state in possible_next_states:
        cost = calculate_cost(state, s, v, a, lane, predictions, goal, MAP)
        costs.append((cost,state))
    _, next_state = min(costs)
    return next_state

def calculate_cost(state, s, v, a, lane, predictions, goal, MAP):
    # cost functions generally designed to output numbers between -1 and 1
    # unless dangerous or illegal behavior
    cost_functions = [
        lane_change_cost,
        velocity_cost,
        gap_size_cost
    ]
    weights = [
        1,
        1,
        1
    ]
    cost = 0
    for cost_func, weight in zip(cost_functions, weights):
        cost += weight * cost_func(state, s,v,a,lane,predictions,goal, MAP)
    return cost

def lane_change_cost(state, s,v,a,lane,predictions,goal, MAP):
    if state not in ["lane_change_left", "lane_change_right"]:
        return 0
    diffs = {"lane_change_left":-1, "lane_change_right":+1}
    next_lane = lane + diffs[state]
    if not MAP.lane_exists_at_location(s, next_lane):
        return DANGEROUS_COST
    if abs(goal.lane - next_lane) < abs(goal.lane - lane):
        return -1
    else:
        return 1

def velocity_cost(state, s,v,a,lane,predictions,goal, MAP):
    speed_limit = MAP.get_speed_limit(s, lane)
    velocity_for_state = get_velocity_for_state(state, s,v,a,lane,predictions,goal,MAP)
    if velocity_for_state > speed_limit: 
        return ILLEGAL_COST
    return (speed_limit - velocity_for_state) / speed_limit # between 0 and 1

def gap_size_cost(state, s,v,a,lane,predictions,goal, MAP):
    L = MAP.vehicle_length
    
    # next evaluate the cost associated with all the available transitions
    
    # COSTS INCLUDE: 
    # 1. Does this make progress to the goal 
    #    in terms of number of lane changes remaining. 
    #    Idea is to penalize lane changes if they aren't necessary.
    # 2. Velocity: We want to drive as close to the speed 
    #    limit as possible. (NOTE: Include speed limit in map)
    # 3. Gap size: when making lane changes we prefer to aim for 
    #    larger gaps than smaller ones.
    # 4. If you wanted to go further you might need a  
    